# 🦌 ELK XELK Transformer Layout Options 🤖

The `XELKTypedLayout` generates a nested dictionary that controls various
[Layout Options](https://www.eclipse.org/elk/reference/options.html) that can be passed
to Elk. Currently only a subset of Layout Options are implemented and not all
interdepenencies are taken into account.

In [ ]:
import json
import pathlib

import ipywidgets
import networkx
import traitlets
from IPython.display import JSON

import ipyelk
import ipyelk.nx
import ipyelk.tools
from ipyelk.diagram import layout_options

In [ ]:
xelk_typed_layout = ipyelk.nx.XELKTypedLayout()

The output `value` of the widget is a nested dictionary. Top level keys are nodes in
networkx. The keys of the next level are `ElkGraphElements` e.g. (`ElkNode`, `ElkLabel`,
...). The last layer are actual Elk layout options key value pairs.

The `XELK` transformer will look to this structure to identify layout options based on
Networkx node and which ElkGraphElements generated from the node.

In [ ]:
JSON(xelk_typed_layout.value)

## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [ ]:
flat_graph = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
)

flat_elk = ipyelk.ElkDiagram()
flat_xelk = ipyelk.nx.XELK(
    source=(flat_graph, None),
    layouts=xelk_typed_layout.value,
)
flat_xelk.connect(flat_elk)


def _element_type_opt_change(change):
    flat_xelk.layouts = xelk_typed_layout.value
    flat_xelk.refresh()


xelk_typed_layout.observe(_element_type_opt_change, "value")
flat_elk

In [ ]:
xelk_typed_layout

### The `ElkJSON` structure passed to `ElkJS` for layout purposes

In [ ]:
from IPython.display import JSON

JSON(flat_xelk.value)

## Hierarchical Diagram with Ports

In [ ]:
hier_tree = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
)
hier_ports = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
)

hier_elk = ipyelk.ElkDiagram()
hier_xelk_layout = ipyelk.nx.XELKTypedLayout()
hier_xelk = ipyelk.nx.XELK(
    source=(hier_ports, hier_tree), layouts=hier_xelk_layout.value
)


def _type_opt_change(change):
    hier_xelk.layouts = hier_xelk_layout.value
    hier_xelk.refresh()


hier_xelk_layout.observe(_type_opt_change, "value")
hier_xelk.connect(hier_elk)


toggle = ipywidgets.Button(description="Toggle Collapsed")


@toggle.on_click
def toggle_node(widget):
    for element_id in hier_elk.selected:
        if element_id in hier_tree:
            for child in hier_tree.neighbors(element_id):
                state = hier_tree.nodes[child].get("hidden", False)
                hier_tree.nodes[child]["hidden"] = not state
            hier_xelk.refresh()


ipywidgets.VBox(
    [
        ipywidgets.HBox(
            [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
        ),
        hier_elk,
        hier_xelk_layout,
    ]
)

In [ ]:
hier_xelk_layout.children = hier_xelk_layout._ui()

## 🦌 Learn More 📖

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)
- [🦌 Interactive ELK App 🕹️](./04_Interactive.ipynb)
- [🦌 Node Label Placement 🏷️](./100_node_label_placement.ipynb)
- [🦌 Text Sizer 📏](./101_text_sizer.ipynb)